In [ ]:
import requests
import datetime
import pandas as pd
import json
import boto3
import http.client

def lambda_handler(event, context):
    api_key = "744a3b85a26ce36af4604f139ec46b9c"
    state_code = ""
    country_code = "ES"
    
    data = {
        "Football Club Name": {
            "Real Madrid": "Madrid",
            "Barcelona": "Barcelona",
            "Girona": "Girona",
            "Atletico Madrid": "Madrid",
            "Athletic Club": "Bilbao",
            "Real Sociedad": "San Sebastian",
            "Real Betis": "Sevilla",
            "Villarreal": "Villarreal",
            "Valencia": "Valencia",
            "Getafe": "Getafe",
            "Alaves": "Vitoria-Gasteiz",
            "Sevilla": "Sevilla",
            "Osasuna": "Pamplona",
            "Las Palmas": "Las Palmas de Gran Canaria",
            "Rayo Vallecano": "Puente de Vallecas",
            "Celta Vigo": "Vigo",
            "Mallorca": "Palma de Mallorca",
            "Cadiz": "Cadiz",
            "Granada CF": "Granada",
            "Almeria": "Almeria"
        }
    }
    
    df = pd.DataFrame(data).reset_index()
    df.columns = ["Football Club Name", "City Name"]
    cities = df['City Name'].drop_duplicates()
    
    prediction_four_days = []
    for city_name in cities:
        url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name},{state_code},{country_code}&limit={1}&appid={api_key}"
        response = requests.get(url)
        data = response.json()
        latitude = data[0]['lat']
        longitude = data[0]['lon']
    
    
        url = f"https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={latitude}&lon={longitude}&appid={api_key}"
        response = requests.get(url)
        data = response.json()
        prediction_four_days.append(data)

    
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M")
    
    # Upload to S3
    s3_bucket_name = "dwlprojectbucket"
    s3_key = f"{formatted_datetime} - predictions4days.json"
    s3 = boto3.client('s3')
    
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(prediction_four_days), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload data to S3')
        }
    
    # INGEST Footballdata --> 1 per hour
    
    api_key = "30548bfbf8msh1742529d7115018p1ffd33jsnb2403e22cd90"
    league_id = 140
    season = '2023'
    conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")
    
    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': api_key
    }
    
    # Make the request to get the fixtures for La Liga in the 2023 season
    conn.request("GET", f"/v3/fixtures?league={league_id}&season={season}", headers=headers)
    
    # Get the response
    res = conn.getresponse()
    data = res.read()
    
    # Decode the response and parse it as JSON
    data_json = json.loads(data.decode("utf-8"))
    
    # Extract the fixtures from the JSON response
    fixtures = data_json['response']
    ns_fixtures = [fixture for fixture in fixtures if fixture['fixture']['status']['short'] == 'NS']
    
    # extraxt the statistics for the fixture
    df = pd.DataFrame(ns_fixtures)
    df['fixture_id'] = df['fixture'].apply(lambda x: x['id'])
    
    i = 0
    lineups_all = []
    for fixture_id in df['fixture_id']:
        i = i + 1
        headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': api_key
        }
    
        conn.request("GET", f"/v3/fixtures/lineups?fixture={fixture_id}", headers=headers)
    
        res = conn.getresponse()
        data = res.read()
    
        lineups_all.append(data.decode("utf-8"))
    
    
    
    # Upload to S3
    s3_bucket_name = "dwlprojectbucket"
    s3_key = f"{formatted_datetime} - lineups.json"
    s3 = boto3.client('s3')
    
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(lineups_all), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload data to S3')
        }
    
    
    return {
        'statusCode': 200,
        'body': json.dumps('Data successfully uploaded to S3 with public read permissions')
    }
